In [1]:
import lightning
import lightning.pytorch.callbacks
from ocd.training import OrderedTrainingModule


8


In [2]:
# set callbacks for the trainer
callbacks = [
    # monitor the learning rate (log to tensorboard)
    lightning.pytorch.callbacks.LearningRateMonitor(logging_interval="epoch"),
]

trainer = lightning.Trainer(
    # accelerator="mps",  # remove this line to run on CPU
    callbacks=callbacks,
    # precision=16, # for mixed precision training
    gradient_clip_val=1.0,
    gradient_clip_algorithm="value",
    track_grad_norm="inf",
    log_every_n_steps=1,
    # detect_anomaly=True,
)


[bnlearn] >Import <asia>
[bnlearn] >Loading bif file </home/hamidreza/Work/myprojects/ocd/venv/lib/python3.10/site-packages/bnlearn/data/asia.bif>
[bnlearn] >Check whether CPDs sum up to one.
[bnlearn] >Check whether CPDs associated with the nodes are consistent: True


In [1]:
# setup data
from ocd.data import CausalDataModule
import dycode
import torch

dycode.register_context(torch)

dm = CausalDataModule(
    name="alarm",  # small dataset asia
    observation_size=2048,  # number of observation samples
    intervention_size=256,  # set to 0 for no intervention
    batch_size=128,
    num_workers=0,  # set to 0 for no multiprocessing
    val_size=0,  # 10% of data for validation, or use int for exact number of samples, set to 0 for no validation
    pin_memory=True,  # set to True for faster data transfer to GPU (if available)
)
dm.setup("fit")


AssertionError: perm must be a permutation of [0, 1, ..., n-1]

In [14]:
# set torch.anomaly_detection(True) to debug
import torch

# Extract the category sizes
in_features = dm.train_data[0].features_values


# torch.autograd.set_detect_anomaly(True)
tm = OrderedTrainingModule(
    in_covariate_features=in_features,
    hidden_features_per_covariate=[
        [128 for i in range(len(in_features))],
        [64 for i in range(len(in_features))],
        [32 for i in range(len(in_features))],
    ],
    batch_norm=False,
    criterion_args=dict(
        terms=[
            "ocd.training.terms.OrderedLikelihoodTerm",
            dict(
                name="norm(gamma)",
                term_function='lambda training_module: training_module.model.Gamma.norm(float("inf"))',
                factor=0,
            ),
            # dict(
            #     name='nothing',
            #     term_function='def term(training_module, batch):\n\ttraining_module.batch=batch\n\treturn torch.zeros(1, device=batch.device)',
            #     factor=0,
            # )
        ]
    ),
    optimizer="torch.optim.AdamW",
    tau_scheduler="lambda training_module: 1 * (0.98 ** training_module.current_epoch if training_module.current_epoch < 1000 else 0.01)",
    lr=0.001,
    scheduler="torch.optim.lr_scheduler.ExponentialLR",
    scheduler_interval="epoch",
    scheduler_args={"gamma": 0.99},
)


tensor([[1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.]])

In [3]:
trainer.fit(tm, dm)
